In [ ]:
import sys
stdout = sys.stdout
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator

import warnings
warnings.simplefilter('ignore')

In [ ]:
#fix output
sys.stdout = stdout
simulator = QasmSimulator()

In [ ]:
def assert_correct_counts(counts, expected_value):
    print('Counts achieved:')
    print(counts)
    keys = counts.keys()
    if expected_value == list(keys)[0]:
        print('✅ Correct prediction 🎉')
    else:
        print('❌ False assertion, maybe there are also more than 1 results! 😥')

def compile_circuit_and_check(circuit, expected_value):
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots=3000)
    result = job.result()
    counts = result.get_counts(circuit)
    assert_correct_counts(counts, expected_value)
    return counts

def check_if_matrix_is_unitary(qubits, matrix):
    circuit = QuantumCircuit(qubits,qubits)
    try:
        circuit.unitary(addition, range(qubits), label="unitary check")
        print('✅ We got an unitary matrix! 🎉')
    except:
        print('❌ Input matrix is not unitary. 😥')
    

# Vector-Operations



# Circuit to add 2 qubits together

Due to the nature of quantum gates, one is not able to simply copy the classical design of a [half-](https://en.wikipedia.org/wiki/Adder_(electronics)#Half_adder) or even [fulladder](https://en.wikipedia.org/wiki/Adder_(electronics)#Full_adder) over to a general quantum computer. This means we have to construct something applicable, that can do the same for us.

## Creating a simple 2 qubit adder

Qubits are usually represented as _vectors_, and the qubit states $\ket{0}$ and $\ket{1}$ are represented as $\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ and $\begin{pmatrix} 0 \\ 1 \end{pmatrix}$, respectively.

When working with more than 1 qubit, we can resemble the statespace as one large vector. With 2 qubits, we have 4 possible states $S$ can be in: "00, 01, 10, 11". This means we can represent the whole state-space with one simple vector:

$$
S = \begin{pmatrix} 0 \\ 1 \\ 0 \\ 0\end{pmatrix}
$$

Where the _1_ in the vector would represent $\ket{01}$.

In this case, we only have one state that would cause any change when doing _addition_. The state to trigger it would be $\ket{11}$, and it's addition would result in $\ket{00}$ with a carry of _1_, which we will ignore for now.

To create a matrix (which we will call $U_{add}$ from now on) that allows the addition of 2 qubits, we can set up equations with our current knowledge:

$$

U_{add} *\ \begin{pmatrix} 1 \\ 0 \\ 0 \\ 0\end{pmatrix} = \begin{pmatrix} 1 \\ 0 \\ 0 \\ 0\end{pmatrix} \\
U_{add} *\ \begin{pmatrix} 0 \\ 1 \\ 0 \\ 0\end{pmatrix} = \begin{pmatrix} 0 \\ 1 \\ 0 \\ 0\end{pmatrix} \\
U_{add} *\ \begin{pmatrix} 0 \\ 0 \\ 1 \\ 0\end{pmatrix} = \begin{pmatrix} 0 \\ 0 \\ 1 \\ 0\end{pmatrix} \\
U_{add} *\ \begin{pmatrix} 0 \\ 0 \\ 0 \\ 1\end{pmatrix} = \begin{pmatrix} 1 \\ 0 \\ 0 \\ 0\end{pmatrix} \\

$$

To find the matrix needed, we can simply multiply the output $O_n$ with the transposed input state $S_n$ to get $U_n$. To create the definitive matrix, we then just add all the resulting matrices together to receive $U_{add}$.

$$
O_0 * S_0^T = U_{0} = \begin{pmatrix} 1 & 0 & 0 & 0\\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0\end{pmatrix} \\
O_1 * S_1^T = U_{1} = \begin{pmatrix} 0 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0\end{pmatrix} \\
O_2 * S_2^T = U_{2} = \begin{pmatrix} 0 & 0 & 0 & 0\\ 0 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0\end{pmatrix} \\
O_3 * S_3^T = U_{3} = \begin{pmatrix} 0 & 0 & 0 & 1\\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0\end{pmatrix} \\
$$

Now we can add up all matrices to get $U_{add}$

$$
U_{add} = \Sigma_{i=0}^{3} U_{i} = \begin{pmatrix} 1 & 0 & 0 & 1\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{pmatrix}

$$

_If you can't figure out why our unitary gate is left of the state, check [this]() out_ <- todo, insert how gates are applied to a qubit

Qiskit offers the possibility to create an unitary gate that can be defined using a matrix. Whilst we could just plug the matrix right in, we have to make sure that the matrix is unitary, as all quantum gates have to be unitary. To check if a matrix is unitary, we have the following equation which has to be fulfilled:

$$

U^{\dagger}U =\ UU^{\dagger} = I\\
U^{\dagger} =\ \overline{U^T}
$$

The _conjugate_ of the matrix would be multiplying all _imaginary constants_ wiht _-1_. As our matrix does not contain any, we can ignore this step. The next one would be to get the transpose of the matrix:

$$

U^T = \begin{pmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 1 & 0 & 0 & 0\end{pmatrix}

$$

From here we can then multiply $U$ with $U^T$ and see whether or not our matrix is unitary

$$

UU^T = \begin{pmatrix} 1 & 0 & 0 & 1\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{pmatrix} *\ \begin{pmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 1 & 0 & 0 & 0\end{pmatrix} =\ \begin{pmatrix} 2 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{pmatrix} \neq \begin{pmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1\end{pmatrix}

$$

As we have shown, the matrix is not unitary. This is also confirmed through Qiskits self-check. 

In [ ]:
# This requires us to import a specific part of the library
import qiskit.quantum_info as qi

addition = qi.Operator([[1, 0, 0, 1],
                        [0, 1, 0, 0],
                        [0, 0, 1, 0],
                        [0, 0, 0, 0]])

check_if_matrix_is_unitary(2, addition)

# Reversible Gates

The reason why this isn't an adequate matrix is because we "lose" the input information. In quantum circuits, gates have to be reversible, which means there cannot be a loss of information. This means, that for our addition, we have to add in _at least_ another qubit which will then be used for the output. This allows the original qubits to retain their state. Due to this, our state vector now changes from a length of 4 to 8 ($|S| = 2^{n}$)

$$
S = \begin{pmatrix} 0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0\end{pmatrix}
$$

With this, we can apply the same calculations as above to retrieve a new $U_{add}$ matrix.

$$
U_{add} = \Sigma_{i=0}^{7} O_{i} *\ S_{i}^T = \begin{pmatrix}
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
        0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\
        0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
        0 & 0 & 0 & 0 & 0 & 0 & 0 &1
        \end{pmatrix}
$$


This is equivalent to the _Toffoli_ Gate, also called a _CCNOT_ Gate. Let's test this with some gates


In [ ]:
addition = qi.Operator([[1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0],
                        [0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 1, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1]])

check_if_matrix_is_unitary(3, addition)

In [ ]:
def generate_circuit(index):
    circuit = QuantumCircuit(3,3)
    if(state_matrix[index][1] == 1):
        circuit.x(0)
    if(state_matrix[index][0] == 1):
        circuit.x(1)
    circuit.barrier()
    circuit.unitary(addition, [0, 1, 2], label='addition')
    circuit.barrier()
    circuit.measure(range(3),range(3)) 
    return circuit

In [ ]:
state_matrix = [[0,0],
                [0,1],
                [1,0],
                [1,1]]

expected_states = [ '000',
                    '101',
                    '110',
                    '011']

for index, expected_state in enumerate(expected_states):
    print("Testing for initial state " + str(state_matrix[index]) + " and expected measurment state " + expected_state)
    circuit = generate_circuit(index)
    display(circuit.draw())
    display(plot_histogram(compile_circuit_and_check(circuit, expected_states[index])))

As we can see, the theory holds up and can be proven using the simulator. The next part would be to increase the amount of addable qubits from 1 + 1 to n + n. This forces us to introduce a $Carry_{out}$ qubit that is separate from the _Out_ qubit, therefor increasing the min. size to 4 qubits. First, let's try to create another unitary matrix to take care of this. As we have 4 qubits now, we have a _16 dimensional state vector_. For $Carry_{in}$ we can reuse our output qubit. Lets create an overview of all the states we are expecting. Note that from above:

$$
q_0 =\ B\\
q_1 =\ A\\
q_3 =\ Out/Carry_{in}\\ 
$$

<center>

| Out/Carry In | A | B | Carry Out | Result |
|:------------:|:-:|:-:|:---------:|--------|
|       0      | 0 | 0 |     0     |   000  |
|       0      | 0 | 1 |     0     |   101  |
|       0      | 1 | 0 |     0     |   110  |
|       0      | 1 | 1 |     1     |   011  |
|       1      | 0 | 0 |     0     |   100  |
|       1      | 0 | 1 |     1     |   001  |
|       1      | 1 | 0 |     1     |   010  |
|       1      | 1 | 1 |     1     |   111  |

</center>

Again, we use the formula from above with $n = 2^4$ to create the matrix

$$
U_{add} = \Sigma_{i=0}^{16} O_{i} *\ S_{i}^T = \begin{pmatrix}
     1  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  1  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  1  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  1  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  1  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  1  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  1  &  0  &  0  &  0 \\
     0  &  1  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  1  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  1  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  1  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  1  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  1  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  1  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  1  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0 \\
     0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  0  &  1 \\
        \end{pmatrix}
$$

It is immediately clear that this matrix cannot be used. The most obvious reason is because the matrix does not offer complete bijection, that is, each input is mapped exactly to one output. What we have here is a _non-injective, non-surjective_ input to output mapping.

This forces us to search for other ways of handling carry over. _What if we use additional gates for it?_
As stated before, we can already handle the output/carry in using our previously constructed matrix. To figure out how to _set_ a carry out qubit, we can consult the table above and see that the carry out is only set when certain conditions apply:

<center>

| Out/Carry In | A | B | Carry Out | Result |
|:------------:|:-:|:-:|:---------:|--------|
|       0      | 1 | 1 |     1     |   011  |
|       1      | 0 | 1 |     1     |   001  |
|       1      | 1 | 0 |     1     |   010  |
|       1      | 1 | 1 |     1     |   111  |

</center>

These conditions can be solved using a _Toffoli_ Gate. It's a double controlled _X_ gate, that _flips_ the third qubit if the previous 2 are in the state $\ket{1}$. With this we can try to solve the table above. For this we need to combine each of the 3 relevant lanes onto the $Carry_{out}$ lane by using _Tiffoli_ gates.

In [ ]:
sub_circuit = QuantumRegister(4)
carry_out = QuantumCircuit(sub_circuit, name="carry over")
carry_out.ccx(0,1,3)
carry_out.ccx(1,2,3)
carry_out.ccx(0,2,3)
carry_out.draw()

In [ ]:
def generate_circuit(index):
    circuit = QuantumCircuit(4,4)
    if(state_matrix[index][2] == 1):
        circuit.x(0)
    if(state_matrix[index][1] == 1):
        circuit.x(1)
    if(state_matrix[index][0] == 1):
        circuit.x(2)
    circuit.barrier()
    circuit.append(carry_out, [0,1,2,3])
    circuit.unitary(addition, [0, 1, 2], label='addition')
    circuit.barrier()
    circuit.measure(range(4),range(4)) 
    return circuit

In [ ]:
state_matrix = [[0,1,1],
                [1,0,1],
                [1,1,0],
                [1,1,1]]

expected_states = [ '1011',
                    '1001',
                    '1010',
                    '1111']

for index, expected_state in enumerate(expected_states):
    print("--------------------------------")
    print("Testing for initial state " + str(state_matrix[index]) + " and expected measurment state " + expected_state)
    circuit = generate_circuit(index)
    display(circuit.draw())
    display(plot_histogram(compile_circuit_and_check(circuit, expected_states[index])))

Now, lets create a silent check to see if this circuit holds true for _all_ cases that were defined in the table in a silent fashion

In [ ]:
state_matrix = [[0,0,0],
                [0,0,1],
                [0,1,0],
                [0,1,1],
                [1,0,0],
                [1,0,1],
                [1,1,0],
                [1,1,1]]

expected_states = [ '0000',
                    '0101',
                    '0110',
                    '1011', 
                    '0100', 
                    '1001',
                    '1010',
                    '1111']

for index, expected_state in enumerate(expected_states):
    print("--------------------------------")
    print("Testing for initial state " + str(state_matrix[index]) + " and expected measurment state " + expected_state)
    circuit = generate_circuit(index)
    compile_circuit_and_check(circuit, expected_states[index])

If we omit the $Carry_{out}$ qubit from a simple 1 bit addition, we only need 3 qubits total. If we have an addition of more than 1 bit, we will need a $Carry_{out}$, but this can be reused for the output of the next bit etc. This results in the following calculations for the amount of qubits needed:

$$
3n = \text{qubits needed}
$$

# Parameterized Gates

To increase the usability of the circuit, we want to be able to set the input value of the qubits. There are two ways of doing so. We can either build the circuit with _X_ gates at the start to set the qubits accordingly, or we can use _parameterized gates_ that are always there and can be set according to the input. On of those gates would be the [_RXGate_](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RXGate.html). Due to those gates being _rotation gates_, we have to work with _pi_ instead of _1_.
First, let's assert what we just said with an example

In [ ]:
atheta = Parameter('aθ')
btheta = Parameter('bθ')
circuit = QuantumCircuit(2,2)
circuit.rx(atheta,0)
circuit.rx(btheta,1)
circuit.barrier()
circuit.measure(range(2), range(2))

states = [  [0,0],
            [0,1],
            [1,0],
            [1,1]]

expected_values = [ '00',
                    '01',
                    '10',
                    '11']

for idx, val in enumerate(states):
    print("--------------------------------")
    print("Testing for initial state " + str(val) + " and expected measurment state " + expected_values[idx])
    circuits = [circuit.bind_parameters([val[1]*np.pi, val[0]*np.pi])]
    display(circuits[-1].draw())
    compile_circuit_and_check(circuits[-1], expected_values[idx])

With this assertion, we can now finally construct a function that generates circuits for us! Let's also encapsulate the sub-circuits here so we have a complete overview:

In [ ]:
def create_parameters(nparameters):
    if nparameters > 123-97:
        raise Exception('The generator is limited to ' + str(123-97-1) + ' bits!')
    else:
        params = []
        for i in range(97, 97+nparameters):
            params.append(Parameter(chr(i) + 'θ'))
    return params

def get_carry_over_circuit():
    sub_circuit = QuantumRegister(4)
    carry_out = QuantumCircuit(sub_circuit, name="carry over")
    carry_out.ccx(0,1,3)
    carry_out.ccx(1,2,3)
    carry_out.ccx(0,2,3)
    return carry_out

def get_adder_circuit():
    return qi.Operator([[1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0],
                        [0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 1, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1]])


def create_full_adder(nbits):
    if nbits < 1:
        raise Exception('You can\'t add less than 1 bit together!')
    n = 3 * nbits + 1
    if n < 4: n = 4
    circuit = QuantumCircuit(n,nbits)
    parameters = create_parameters(2*nbits)
    iterparam = parameters.copy()
    circuit.rx(iterparam[0],0)
    circuit.rx(iterparam[1],1)
    del iterparam[0:2]
    iterable_parameters = iter(iterparam)
    for i in range(4,n):
        if i%3 == 0:
            continue
        else:
            circuit.rx(next(iterable_parameters),i)
    circuit.barrier()
    #start the circuitry

    for i in range(nbits):
        if i == 0:
            circuit.append(get_carry_over_circuit(), [0, 1, 2, 3])
            circuit.unitary(get_adder_circuit(), [0, 1, 2], label='addition')
            circuit.measure(2,0)
        else:
            circuit.append(get_carry_over_circuit(), [i*3+1, i*3+2, i*3, i*3+3])
            circuit.unitary(get_adder_circuit(), [i*3+1, i*3+2, i*3], label='addition')
            circuit.measure(i*3,i)

    return circuit

#drawin a max sized adder. max size is limited due to the alphabet not having more letters :)
create_full_adder(4).draw()

Now let's create tests to make sure we receive the expected result.

In [48]:
# qubits to simulate and test:
n = 3
# Attention: going to 5 or above can lead to enormous waiting times or even crash the python kernel


def compile_circuit_and_test(circuit, expected_value):
    result = False
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots=3000)
    result = job.result()
    counts = result.get_counts(circuit)
    keys = counts.keys()
    if len(keys) == 1 and expected_value == list(keys)[0]:
        result = True
    return result, list(keys)[0]

def turn_binary_strings_to_parameters(n, A, B):
    Aiter = iter(A.replace('0b','')[::-1])
    Biter = iter(B.replace('0b','')[::-1])
    result = bin(int(A,2)+int(B,2)).replace('0b','')
    params = np.zeros(n*2)
    for i in range(0,n*2,2):
        params[i] = int(next(Aiter, 0))*np.pi
        params[i+1] = int(next(Biter, 0))*np.pi
    return params, result

def format_binary_to_length(n, binary):
    while len(binary) > n:
        binary = binary[1:]
    while len(binary) < n:
        binary = '0' + binary
    return binary

results = []
circuit = create_full_adder(n)

for i in range(2**n):
    A = bin(i).replace('0b','')
    for j in range(2**n):
        B = bin(j).replace('0b','')
        Aiter = iter(A[::-1])
        Biter = iter(B[::-1])
        A = format_binary_to_length(n, A)
        B = format_binary_to_length(n, B)
        params, C = turn_binary_strings_to_parameters(n, A, B)
        C = format_binary_to_length(n, C)
        qc = circuit.bind_parameters(params)
        result, Ccalculated = compile_circuit_and_test(qc, C)
    
        results.append([A,B,C,Ccalculated, '✅' if result else '❌'])


from IPython.display import display, Markdown, Latex
display(Markdown('|Ain | Bin | Cexpected | Ccalculated | Result |\n'+'|:---:|:---:|:-----:|:------:|--------|\n'+''.join(['|'+val[0]+'|'+val[1]+'|'+val[2]+'|'+val[3]+'|'+val[4]+'|\n' for val in results])))

|Ain | Bin | Cexpected | Ccalculated | Result |
|:---:|:---:|:-----:|:------:|--------|
|000|000|000|000|✅|
|000|001|001|001|✅|
|000|010|010|010|✅|
|000|011|011|011|✅|
|000|100|100|100|✅|
|000|101|101|101|✅|
|000|110|110|110|✅|
|000|111|111|111|✅|
|001|000|001|001|✅|
|001|001|010|010|✅|
|001|010|011|011|✅|
|001|011|100|100|✅|
|001|100|101|101|✅|
|001|101|110|110|✅|
|001|110|111|111|✅|
|001|111|000|000|✅|
|010|000|010|010|✅|
|010|001|011|011|✅|
|010|010|100|100|✅|
|010|011|101|101|✅|
|010|100|110|110|✅|
|010|101|111|111|✅|
|010|110|000|000|✅|
|010|111|001|001|✅|
|011|000|011|011|✅|
|011|001|100|100|✅|
|011|010|101|101|✅|
|011|011|110|110|✅|
|011|100|111|111|✅|
|011|101|000|000|✅|
|011|110|001|001|✅|
|011|111|010|010|✅|
|100|000|100|100|✅|
|100|001|101|101|✅|
|100|010|110|110|✅|
|100|011|111|111|✅|
|100|100|000|000|✅|
|100|101|001|001|✅|
|100|110|010|010|✅|
|100|111|011|011|✅|
|101|000|101|101|✅|
|101|001|110|110|✅|
|101|010|111|111|✅|
|101|011|000|000|✅|
|101|100|001|001|✅|
|101|101|010|010|✅|
|101|110|011|011|✅|
|101|111|100|100|✅|
|110|000|110|110|✅|
|110|001|111|111|✅|
|110|010|000|000|✅|
|110|011|001|001|✅|
|110|100|010|010|✅|
|110|101|011|011|✅|
|110|110|100|100|✅|
|110|111|101|101|✅|
|111|000|111|111|✅|
|111|001|000|000|✅|
|111|010|001|001|✅|
|111|011|010|010|✅|
|111|100|011|011|✅|
|111|101|100|100|✅|
|111|110|101|101|✅|
|111|111|110|110|✅|
